In [ ]:
%pip install transformers torch datasets icecream accelerate trl --no-cache-dir -U

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
from trl import SFTTrainer
from datasets import load_dataset
from datasets.arrow_dataset import Dataset
from icecream import ic

# Carregar o modelo e o tokenizador GPT-2
model_name = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Definir um token de preenchimento
tokenizer.pad_token = tokenizer.eos_token

ic.configureOutput(prefix="Model confgi")
ic(model)

# Carregar o dataset de diálogos
#dataset = load_dataset("Osondu/reddit_autism_dataset", trust_remote_code=True )
#dataset_autism = load_dataset("Osondu/reddit_autism_dataset", split="train" )
# Verificar a estrutura do dataset


In [ ]:
def gen_batches_train():
    dataset = load_dataset("Osondu/reddit_autism_dataset",  streaming=True,  split="train")
    ic.configureOutput(prefix="Dataset autism\n")
    ic(dataset)

    total_samples = 800
    val_pct = 0.1
    train_limit = int(total_samples * (1 - val_pct))
    counter = 0

    for row in dataset:
        if counter >= train_limit:
            break

        inst = row["instructions"]
        input = row["input"]
        output = row["output"]

        chat_template = f"<s>[INST] {inst}\nInput: {input} [/INST] {output}</s>"
        tokenized_output = tokenizer(chat_template)
        yield {'text': chat_template}
        counter += 1


def gen_batches_val():
    dataset = load_dataset("Osondu/reddit_autism_dataset",  streaming=True,  split="train")
    total_samples = 800
    val_pct = 0.1
    train_limit = int(total_samples * (1 - val_pct))
    counter = 0

    for row in dataset:
        if counter >= total_samples:
           break

        inst = row["instructions"]
        input = row["input"]
        output = row["output"]

        chat_template = f"<s>[INST] {inst}\nInput: {input} [/INST] {output}</s>"
        tokenized_output = tokenizer(chat_template)
        yield {'text': chat_template}
        counter += 1